In [1]:
# ============================================================
# QF5210 - Momentum Strategies (Single & Multi Asset)
# Data: PPXDM00 (e.g., crack spread) & PPXDK00 (e.g., price)
# Return: Percentage return (ΔP/P), next-day execution (shifted signal)
# Splits: Train (<= 2025-07-31), Val (2025-08-01~08-31), Test (2025-09-01~09-15)
# Output: CSVs + PNG plots under OUTPUT_DIR
# ============================================================

import os
import re
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

# ========= 需要修改为你的本地路径 =========
FILE_PPXDM = r"C:\Users\chaiy\Desktop\Platts_PPXDM00.csv"
FILE_PPXDK = r"C:\Users\chaiy\Desktop\Platts_PPXDK00.csv"  
OUTPUT_DIR = r"C:\Users\chaiy\Desktop\ppx_combo_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ========= 课程要求的时间切分 =========
TRAIN_END  = pd.Timestamp("2025-07-31")
VAL_START  = pd.Timestamp("2025-08-01")
VAL_END    = pd.Timestamp("2025-08-31")
TEST_START = pd.Timestamp("2025-09-01")
TEST_END   = pd.Timestamp("2025-09-15")

# ========= 网格搜索空间（可调） =========
MA_SHORTS = [3, 5, 7, 10, 12]
MA_LONGS  = [15, 20, 30, 45]
MOM_LB    = [5, 10, 15, 20]    # momentum lookback (sign of price change vs L days ago)
VOL_Q     = 0.50               # 波动过滤分位数阈值（0~1），默认0.5；可调0.6/0.7更稳

# ------------------------------------------------------------
# 1) 读取函数：自动识别日期列与数值列（鲁棒）
# ------------------------------------------------------------
def load_platts_csv(path: str) -> pd.DataFrame:
    """
    返回 DataFrame(index=date, columns=['value'])，日期升序去重。
    自动尝试多种表头偏移与格式。
    """
    for skip in [0, 1, 2, 3, 4, 5]:
        try:
            df = pd.read_csv(path, skiprows=skip)
            df = df.dropna(axis=1, how='all')
            if df.empty:
                continue

            # 情形A：有 'Timestamp' 列
            if 'Timestamp' in df.columns:
                num_cols = [c for c in df.columns if c != 'Timestamp']
                for c in num_cols:
                    df[c] = pd.to_numeric(df[c], errors='coerce')
                if num_cols:
                    value_col = max(num_cols, key=lambda c: df[c].notna().sum())
                    out = df[['Timestamp', value_col]].copy()
                    out.columns = ['date', 'value']
                    out['date'] = pd.to_datetime(out['date'], errors='coerce')
                    out = (out.dropna(subset=['date', 'value'])
                              .sort_values('date')
                              .drop_duplicates('date'))
                    return out.set_index('date')[['value']]

            # 情形B：寻找“像日期”的列 + 一个数值列
            parsed_dates = {}
            for c in df.columns:
                d = pd.to_datetime(df[c], errors='coerce', dayfirst=False)
                if d.notna().sum() > len(df) * 0.6:
                    parsed_dates[c] = d
            if parsed_dates:
                date_col = max(parsed_dates, key=lambda c: parsed_dates[c].notna().sum())
                num_cols = [c for c in df.columns if c != date_col]
                for c in num_cols:
                    df[c] = pd.to_numeric(df[c], errors='coerce')
                if num_cols:
                    value_col = max(num_cols, key=lambda c: df[c].notna().sum())
                    out = pd.DataFrame({
                        'date': pd.to_datetime(df[date_col], errors='coerce'),
                        'value': df[value_col]
                    })
                    out = (out.dropna(subset=['date', 'value'])
                              .sort_values('date')
                              .drop_duplicates('date'))
                    return out.set_index('date')[['value']]
        except Exception:
            continue

    # 情形C：逐行正则兜底（当 CSV 极不规则）
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
    data = []
    for line in lines:
        m = re.search(r'(\d{4}-\d{2}-\d{2}).*?(-?\d+(?:\.\d+)?)\s*$', line.strip())
        if m:
            try:
                dt  = datetime.strptime(m.group(1), "%Y-%m-%d")
                val = float(m.group(2))
                data.append((dt, val))
            except Exception:
                pass
    out = pd.DataFrame(data, columns=['date', 'value'])
    if out.empty:
        raise ValueError(f"无法从文件解析数据: {path}")
    out = (out.dropna()
              .sort_values('date')
              .drop_duplicates('date')
              .set_index('date'))
    return out[['value']]

# ------------------------------------------------------------
# 2) 基础工具：分期、绘图
# ------------------------------------------------------------
def split_by_course(df: pd.DataFrame):
    train = df.loc[:TRAIN_END].copy()
    val   = df.loc[VAL_START:VAL_END].copy()
    test  = df.loc[TEST_START:TEST_END].copy()
    return train, val, test

def plot_cumret_pct(returns: pd.Series, title: str, save_path: str):
    eq = (1 + returns.fillna(0)).cumprod() - 1
    plt.figure(figsize=(8, 3.5))
    plt.plot(eq.index, eq.values * 100)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Cumulative Return (%)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

# ------------------------------------------------------------
# 3) 信号族：MA 交叉 / 纯动量符号 + 可选波动过滤
# ------------------------------------------------------------
def sig_ma_crossover(df: pd.DataFrame, s: int, l: int) -> pd.Series:
    x = df.copy()
    x['MA_s'] = x['value'].rolling(s, min_periods=s).mean()
    x['MA_l'] = x['value'].rolling(l, min_periods=l).mean()
    sig = np.where(x['MA_s'] > x['MA_l'], 1, -1)
    return pd.Series(sig, index=x.index, name='signal')

def sig_momentum(df: pd.DataFrame, lookback: int) -> pd.Series:
    x = df.copy()
    mom = x['value'] / x['value'].shift(lookback) - 1
    sig = np.sign(mom).replace(0, np.nan).ffill().fillna(0)
    return pd.Series(sig, index=x.index, name='signal')

def vol_mask(df: pd.DataFrame, window=10, q=VOL_Q) -> pd.Series:
    r = df['value'].pct_change()
    vol = r.rolling(window).std()
    thr = vol.rolling(window).quantile(q)
    return (vol > thr).fillna(False)

def run_strategy(df: pd.DataFrame, signal: pd.Series,
                 use_vol_filter=False, vol_window=10, vol_q=VOL_Q) -> pd.Series:
    sig = signal.copy()
    if use_vol_filter:
        mask = vol_mask(df, vol_window, vol_q)
        sig = sig.where(mask, 0)  # 过滤低波动期，空仓
    ret = df['value'].pct_change().fillna(0)
    strat = sig.shift(1).fillna(0) * ret   # 次日执行，避免未来函数
    return strat

# ------------------------------------------------------------
# 4) 指标（ΔP/P 百分比口径）
# ------------------------------------------------------------
def metrics_from_returns(strat: pd.Series) -> dict:
    pnl = strat.fillna(0)
    mu = pnl.mean()
    sd = pnl.std(ddof=1)
    ann_ret = (1 + mu)**252 - 1 if len(pnl) else np.nan
    ann_vol = sd * np.sqrt(252) if sd > 0 else np.nan
    sharpe  = (mu / sd * np.sqrt(252)) if sd and sd > 0 else np.nan
    eq = (1 + pnl).cumprod()
    dd = eq / eq.cummax() - 1
    mdd = dd.min() if len(dd) else np.nan
    total = eq.iloc[-1] - 1 if len(eq) else np.nan
    return {
        "Annual Return (%)": ann_ret * 100 if pd.notna(ann_ret) else np.nan,
        "Annual Volatility (%)": ann_vol * 100 if pd.notna(ann_vol) else np.nan,
        "Sharpe Ratio": sharpe,
        "Max Drawdown (%)": mdd * 100 if pd.notna(mdd) else np.nan,
        "Total Return (%)": total * 100 if pd.notna(total) else np.nan
    }

# ------------------------------------------------------------
# 5) 验证期选参（单资产）
# ------------------------------------------------------------
def validate_on_val(df_tr: pd.DataFrame, df_va: pd.DataFrame) -> pd.DataFrame:
    rows = []

    # MA 交叉
    for s, l in product(MA_SHORTS, MA_LONGS):
        if s >= l:
            continue
        sig = sig_ma_crossover(df_va, s, l)
        # 无过滤
        strat = run_strategy(df_va, sig, use_vol_filter=False)
        m = metrics_from_returns(strat); m.update({"family": "MA", "short": s, "long": l, "vol_filter": False})
        rows.append(m)
        # 加过滤
        strat2 = run_strategy(df_va, sig, use_vol_filter=True, vol_window=10, vol_q=VOL_Q)
        m2 = metrics_from_returns(strat2); m2.update({"family": "MA+Vol", "short": s, "long": l, "vol_filter": True})
        rows.append(m2)

    # 纯动量符号
    for L in MOM_LB:
        sig = sig_momentum(df_va, L)
        strat = run_strategy(df_va, sig, use_vol_filter=False)
        m = metrics_from_returns(strat); m.update({"family": "MOM", "lookback": L, "vol_filter": False})
        rows.append(m)
        strat2 = run_strategy(df_va, sig, use_vol_filter=True, vol_window=10, vol_q=VOL_Q)
        m2 = metrics_from_returns(strat2); m2.update({"family": "MOM+Vol", "lookback": L, "vol_filter": True})
        rows.append(m2)

    res = pd.DataFrame(rows).sort_values(["Sharpe Ratio", "Annual Return (%)"],
                                         ascending=[False, False]).reset_index(drop=True)
    return res

def pick_best(res_df: pd.DataFrame) -> dict:
    if res_df.empty:
        raise RuntimeError("验证期结果为空；请检查数据是否涵盖 2025-08。")
    return res_df.iloc[0].to_dict()

def build_signal_by_best(df: pd.DataFrame, best: dict) -> pd.Series:
    fam = str(best["family"])
    if "MA" in fam:
        s = int(best["short"]); l = int(best["long"])
        sig = sig_ma_crossover(df, s, l)
        use_vol = ("Vol" in fam)
    else:
        L = int(best["lookback"])
        sig = sig_momentum(df, L)
        use_vol = ("Vol" in fam)
    return sig, use_vol

def evaluate_phases(df_tr, df_va, df_te, best: dict):
    sig_tr, use_vol = build_signal_by_best(df_tr, best)
    sig_va, _ = build_signal_by_best(df_va, best)  # same family/params
    sig_te, _ = build_signal_by_best(df_te, best)
    st_tr = run_strategy(df_tr, sig_tr, use_vol_filter=use_vol)
    st_va = run_strategy(df_va, sig_va, use_vol_filter=use_vol)
    st_te = run_strategy(df_te, sig_te, use_vol_filter=use_vol)
    return (st_tr, metrics_from_returns(st_tr)), \
           (st_va, metrics_from_returns(st_va)), \
           (st_te, metrics_from_returns(st_te)), \
           use_vol

# ------------------------------------------------------------
# 6) 多资产组合：共识法 & 加权法（验证期上选 w，测试期锁参）
# ------------------------------------------------------------
def build_common_index(dm: pd.DataFrame, dk: pd.DataFrame):
    common_idx = dm.index.intersection(dk.index)
    return dm.loc[common_idx].copy(), dk.loc[common_idx].copy()

def combine_consensus(dm_df, dk_df, sig_dm, sig_dk):
    idx = dm_df.index.union(dk_df.index)
    sig_dm_s = sig_dm.reindex(idx).fillna(0)
    sig_dk_s = sig_dk.reindex(idx).fillna(0)
    agree = (np.sign(sig_dm_s) == np.sign(sig_dk_s)) & (sig_dm_s != 0)
    pos = pd.Series(0.0, index=idx)
    pos.loc[agree] = np.sign(sig_dm_s.loc[agree])
    ret = (dm_df['value'].reindex(idx).pct_change().fillna(0) +
           dk_df['value'].reindex(idx).pct_change().fillna(0)) / 2.0
    strat = pos.shift(1).fillna(0) * ret
    return strat

def combine_weighted(dm_df, dk_df, sig_dm, sig_dk, w: float):
    idx = dm_df.index.union(dk_df.index)
    pos = np.sign(w * sig_dm.reindex(idx).fillna(0) + (1 - w) * sig_dk.reindex(idx).fillna(0))
    ret = (dm_df['value'].reindex(idx).pct_change().fillna(0) +
           dk_df['value'].reindex(idx).pct_change().fillna(0)) / 2.0
    strat = pos.shift(1).fillna(0) * ret
    return strat

def sharpe_of(r: pd.Series) -> float:
    r = r.fillna(0)
    mu = r.mean(); sd = r.std(ddof=1)
    return (mu / sd * np.sqrt(252)) if sd > 0 else -np.inf

def main():
    # 读取数据
    dm = load_platts_csv(FILE_PPXDM).sort_index()
    dk = load_platts_csv(FILE_PPXDK).sort_index()

    # 分期（单资产）
    dm_tr, dm_va, dm_te = split_by_course(dm)
    dk_tr, dk_va, dk_te = split_by_course(dk)

    # 验证期选参（单资产）
    dm_val_res = validate_on_val(dm_tr, dm_va)
    dk_val_res = validate_on_val(dk_tr, dk_va)

    dm_val_res.to_csv(os.path.join(OUTPUT_DIR, "PPXDM00_validation_grid_all.csv"), index=False)
    dk_val_res.to_csv(os.path.join(OUTPUT_DIR, "PPXDK00_validation_grid_all.csv"), index=False)

    dm_best = pick_best(dm_val_res)
    dk_best = pick_best(dk_val_res)

    # 锁参评估（单资产）
    dm_tr_pack, dm_va_pack, dm_te_pack, dm_use_vol = evaluate_phases(dm_tr, dm_va, dm_te, dm_best)
    dk_tr_pack, dk_va_pack, dk_te_pack, dk_use_vol = evaluate_phases(dk_tr, dk_va, dk_te, dk_best)

    # 单资产测试汇总
    single_summary = pd.DataFrame([
        {"Strategy": f"PPXDM00_Test ({dm_best['family']})", **dm_te_pack[1]},
        {"Strategy": f"PPXDK00_Test ({dk_best['family']})", **dk_te_pack[1]},
    ])
    single_summary.to_csv(os.path.join(OUTPUT_DIR, "SingleAsset_Test_Summary.csv"), index=False)

    # 绘图（测试期累计收益）
    plot_cumret_pct(dm_te_pack[0], f"PPXDM00 — Test Cumulative Return (%)  [{dm_best['family']}]", 
                    os.path.join(OUTPUT_DIR, "PPXDM00_Test_cumret.png"))
    plot_cumret_pct(dk_te_pack[0], f"PPXDK00 — Test Cumulative Return (%)  [{dk_best['family']}]", 
                    os.path.join(OUTPUT_DIR, "PPXDK00_Test_cumret.png"))

    # 多资产（仅在公共日期上组合）
    dm_c, dk_c = build_common_index(dm, dk)
    dm_c_tr, dm_c_va, dm_c_te = split_by_course(dm_c)
    dk_c_tr, dk_c_va, dk_c_te = split_by_course(dk_c)

    # 用“单资产验证期最佳家族/参数”在公共区间上构建信号
    dm_sig_va, _ = build_signal_by_best(dm_c_va, dm_best)
    dk_sig_va, _ = build_signal_by_best(dk_c_va, dk_best)
    dm_sig_te, _ = build_signal_by_best(dm_c_te, dm_best)
    dk_sig_te, _ = build_signal_by_best(dk_c_te, dk_best)

    # 验证期上优化加权系数 w ∈ {0.0, 0.1, ..., 1.0}
    best_w, best_ws = 0.5, -np.inf
    for w in [i/10 for i in range(11)]:
        strat_w_val = combine_weighted(dm_c_va, dk_c_va, dm_sig_va, dk_sig_va, w)
        s = sharpe_of(strat_w_val)
        if s > best_ws:
            best_w, best_ws = w, s

    # 测试期组合：共识法 & 加权法(锁定 best_w)
    cons_strat_te = combine_consensus(dm_c_te, dk_c_te, dm_sig_te, dk_sig_te)
    wgt_strat_te  = combine_weighted(dm_c_te, dk_c_te, dm_sig_te, dk_sig_te, best_w)

    combo_summary = pd.DataFrame([
        {"Strategy": "Combined Consensus (Test)", **metrics_from_returns(cons_strat_te)},
        {"Strategy": f"Combined Weighted w={best_w:.1f} (Test)", **metrics_from_returns(wgt_strat_te)},
    ])
    combo_summary.to_csv(os.path.join(OUTPUT_DIR, "Combined_Test_Summary.csv"), index=False)

    plot_cumret_pct(cons_strat_te, "Combined Consensus — Test Cumulative Return (%)",
                    os.path.join(OUTPUT_DIR, "Combined_Consensus_Test_cumret.png"))
    plot_cumret_pct(wgt_strat_te,  f"Combined Weighted (w={best_w:.1f}) — Test Cumulative Return (%)",
                    os.path.join(OUTPUT_DIR, "Combined_Weighted_Test_cumret.png"))

    # 控制台摘要
    print("\n================== Validation Best ==================")
    print("PPXDM00 Best:", dm_best)
    print("PPXDK00 Best:", dk_best)
    print("\n================== Test Summary (Single) ============")
    print(single_summary.to_string(index=False))
    print("\n================== Test Summary (Combined) ==========")
    print(combo_summary.to_string(index=False))
    print("\nFiles saved to:", OUTPUT_DIR)

if __name__ == "__main__":
    main()



================== Validation Best ==================
PPXDM00 Best: {'Annual Return (%)': 470.55839887480033, 'Annual Volatility (%)': 157.93769460704462, 'Sharpe Ratio': 1.1064340010587577, 'Max Drawdown (%)': -43.893884184796185, 'Total Return (%)': 4.908797206288962, 'family': 'MA', 'short': 3.0, 'long': 15.0, 'vol_filter': False, 'lookback': nan}
PPXDK00 Best: {'Annual Return (%)': 135.0163917779569, 'Annual Volatility (%)': 17.981019752405043, 'Sharpe Ratio': 4.7602162083508, 'Max Drawdown (%)': -3.743400995045343, 'Total Return (%)': 7.243772524665726, 'family': 'MA', 'short': 5.0, 'long': 15.0, 'vol_filter': False, 'lookback': nan}

================== Test Summary (Single) ============
         Strategy  Annual Return (%)  Annual Volatility (%)  Sharpe Ratio  Max Drawdown (%)  Total Return (%)
PPXDM00_Test (MA)          -9.627297             161.354430     -0.062724        -29.336300         -5.541707
PPXDK00_Test (MA)          38.845378              22.707185      1.446258    